In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sqlite3 as lite
import datetime as dt
from datetime import datetime
from datetime import timedelta
from datetime import date
import time as time
import matplotlib.pyplot as plt
import seaborn as sb
import fredapi as fa
import re
import csv
import yfinance as yf
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
import statistics
import plotly.graph_objects as go

Get beige book urls 

In [2]:
html_link = pd.DataFrame(columns=["Date", "URL"])
x = 0
for year in range(1996, 2023):
    master_url = "https://www.federalreserve.gov/monetarypolicy/beigebook" + str(year) + ".htm"
    print("Processing Beige Book Year: " + str(year) + " via " + master_url)
    time.sleep(1 + 0.06 * np.random.rand())

    page = requests.get(master_url)
    soup = BeautifulSoup(page.content, 'html.parser')
    results = soup.find('table', class_='table table-layout')
    hrefs = results.find_all('a', href=True)
    for href in hrefs:
        if href.has_attr('href') and ('.htm' in href['href']):

            date_url = ((re.findall('([0-9]{6,})', href['href']))[0] + "01")[:8]
            date_report = datetime.strptime(date_url, '%Y%m%d').date()

            if 1996 <= year <= 2010:
                html_link.loc[x, ['Date', 'URL']] = [date_report, str(href['href']).replace("default", "FullReport")]
            elif 2011 <= year <= 2016:
                html_link.loc[x, ['Date', 'URL']] = [date_report, href['href']]
            if year >= 2017:
                html_link.loc[x, ['Date', 'URL']] = [date_report, str("https://www.federalreserve.gov") + str(href['href'])]

            x = x + 1
# html_link.to_csv('html.csv')

Processing Beige Book Year: 1996 via https://www.federalreserve.gov/monetarypolicy/beigebook1996.htm
Processing Beige Book Year: 1997 via https://www.federalreserve.gov/monetarypolicy/beigebook1997.htm
Processing Beige Book Year: 1998 via https://www.federalreserve.gov/monetarypolicy/beigebook1998.htm
Processing Beige Book Year: 1999 via https://www.federalreserve.gov/monetarypolicy/beigebook1999.htm
Processing Beige Book Year: 2000 via https://www.federalreserve.gov/monetarypolicy/beigebook2000.htm
Processing Beige Book Year: 2001 via https://www.federalreserve.gov/monetarypolicy/beigebook2001.htm
Processing Beige Book Year: 2002 via https://www.federalreserve.gov/monetarypolicy/beigebook2002.htm
Processing Beige Book Year: 2003 via https://www.federalreserve.gov/monetarypolicy/beigebook2003.htm
Processing Beige Book Year: 2004 via https://www.federalreserve.gov/monetarypolicy/beigebook2004.htm
Processing Beige Book Year: 2005 via https://www.federalreserve.gov/monetarypolicy/beigeboo

Scraping functions for 1996-2010 beige books


In [3]:
def get_paragraph_text(bs_tag):
    parent_body = str(bs_tag.parent)
    result = ''
    if parent_body.find(str(bs_tag)):
        block_after_strong = parent_body[parent_body.find(str(bs_tag)) + len(str(bs_tag)):]
        result = BeautifulSoup(block_after_strong[:block_after_strong.find('<strong>')],
                               'html.parser').get_text().strip()

    return result


def parse_and_strip(line_to_strip):
    try:
        soup_response = BeautifulSoup(' '.join(line_to_strip.split()), 'html.parser')
    except:
        soup_response = BeautifulSoup(' '.join(line_to_strip.split()).replace('/', ' '), 'html.parser')
    return soup_response.get_text().strip()


def save_line(url_date, region, sector, comment):
    return [url_date, region.replace(" ", "_").replace(".", ""), sector, comment]


def process_url_1996_2010(df, process_year, report_date, url):
    print(url)
    url_date = (re.findall('([0-9]{8})', url))[0]
    full_report = requests.get(url)
    soup_response = BeautifulSoup(full_report.content, 'html.parser')
    if 1996 <= process_year < 2011:
        tables = soup_response.find_all('table')
        for table in tables:
            tds = table.find_all('td')
            if len(tds) == 1:
                scope = ''
                topic_name = ''
                paragraph = ''

                strongs = str(tds).split('<strong>')
                for strong in strongs:
                    str_strong = str(strong)
                    str_strip = parse_and_strip(str_strong)
                    if '<td>' in str_strong:
                        pass
                    elif '<font size="+1">' in str_strong:
                        if scope != '':
                            df.loc[len(df), ["Date", "Region", "Sector", "Comments"]] = \
                                save_line(report_date, "_" + scope, topic_name, paragraph)

                        district_pos = str_strong[:str_strong.find('</strong>')].find('District')
                        if district_pos > 0:
                            scope = parse_and_strip(str_strong[district_pos + 8:str_strong.find('</strong>')]).replace('-', '')
                        else:
                            scope = str_strip  # parse_and_strip(strong)
                        topic_name = 'Summary'
                        paragraph = parse_and_strip(str_strong[str_strong.find('</strong>') + 9:])
                    elif str_strong.find('</strong>') > 100:
                        paragraph = paragraph + ' ' + ' '.join(str_strip.split())
                    else:
                        df.loc[len(df), ["Date", "Region", "Sector", "Comments"]] = \
                            save_line(report_date, "_" + scope, topic_name, paragraph)
                        end_strong = str_strong.find('</strong>')
                        topic_name = ' '.join(str_strong[:end_strong].split())
                        paragraph = ' '.join(parse_and_strip(str_strong[end_strong:]).split())

                if scope != '':
                    df.loc[len(df), ["Date", "Region", "Sector", "Comments"]] = \
                        save_line(report_date, "_" + scope, topic_name, paragraph)

Code to get Beige Book text


Beige book scraping 

In [4]:
#Region mapping
page_count = 0
beigebook = pd.DataFrame(columns=["Date", "Region", "Sector", "Comments"])

count = 0
region_list = ["div_summary", "div_boston", "div_new_york", "div_philadelphia", "div_cleveland",
               "div_richmond", "div_atlanta", "div_chicago", "div_st_louis", "div_minneapolis", "div_kansas_city",
               "div_dallas", "div_san_francisco"]
row = -1

In [5]:
#Scraping bit
for index, item in html_link.iterrows():
    date = item["Date"]
    date_object = datetime.strptime(str(date), "%Y-%m-%d")
    year = date_object.year

    response1 = requests.get(item["URL"])
    soup2 = BeautifulSoup(response1.content, 'html.parser')
    region_count = 0
    # Code to scrape data for period between 2011-2017
    # Creating a loop to iterate over the different region sections in the HTML
    if 1996 <= year <= 2010:
        process_url_1996_2010(beigebook, year, item["Date"], item["URL"])
    elif 2011 <= year < 2017:
        for region in region_list:
            region_count = 0
            print(region_count)
            print("Processing Beige Book for :" + str(region) + " " + str(date))
            location = soup2.find('div', {'id': region})
            ptags = location.find_all('p')

            comms = ""
            sector_name = "Overall"

            strong_status = 0  # Binary toggle to assess if the p tags are within a strong tag

            # Creating a loop to iterate over the elements in the Div Tag / Region section
            if ptags is not None:
                for elements in ptags:

                    strong_tag = elements.find("strong")

                    if strong_tag is not None:
                        row = row + 1
                        strong_status = 1
                        sector_name = strong_tag.text
                        comms = elements.text
                    else:
                        strong_status = 0
                        comms = comms + elements.text + " "
                    beigebook.loc[len(beigebook), ["Date", "Region", "Sector", "Comments"]] = [date, str(region), sector_name,
                                                                                    comms]
    elif year >= 2017:
        location = soup2.find('div', {'id': "article"})
        tags = ['p', 'a', "h4"]
        ptags = location.find_all(tags)

        comms = ""
        sector_name = "Overall"
        region = region_list[region_count]
        # year_list_item = year_list_item +1

        print("Processing Beige Book for :" + str(region) + " " + str(date))

        for elements in ptags:

            # print("Processing Beige Book for :" +str(region)+" "+str(date))
            strong_tag = elements.find("strong")
            a_tag = elements.find("h4")

            #  To run a check on h4 tag so that the region variable is updated in the dataframe

            if strong_tag is not None:
                row = row + 1
                strong_status = 1
                sector_name = strong_tag.text
                comms = elements.text

            if elements.name == "h4":
                if elements.text == "Highlights by Federal Reserve District":
                    region = region_list[0]
                if elements.text == "Federal Reserve Bank of Boston":
                    region = region_list[0]
                    region_count = 1
                if elements.text == "Federal Reserve Bank of New York":
                    region = region_list[1]
                    region_count = 2
                if elements.text == "Federal Reserve Bank of Philadelphia":
                    region = region_list[2]
                    region_count = 3
                if elements.text == "Federal Reserve Bank of Cleveland":
                    region = region_list[3]
                    region_count = 4
                if elements.text == "Federal Reserve Bank of Richmond":
                    region = region_list[4]
                    region_count = 5
                if elements.text == "Federal Reserve Bank of Atlanta":
                    region = region_list[5]
                    region_count = 6
                if elements.text == "Federal Reserve Bank of Chicago":
                    region = region_list[6]
                    region_count = 7
                if elements.text == "Federal Reserve Bank of St. Louis":
                    region = region_list[7]
                    region_count = 8
                if elements.text == "Federal Reserve Bank of Minneapolis":
                    region = region_list[8]
                    region_count = 9
                if elements.text == "Federal Reserve Bank of Kansas City":
                    region = region_list[9]
                    region_count = 10
                if elements.text == "Federal Reserve Bank of Dallas":
                    region = region_list[10]
                    region_count = 11
                if elements.text == "Federal Reserve Bank of San Francisco":
                    region = region_list[11]
                    region_count = 12
                print(elements.text)
                print(region)

            else:
                strong_status = 0
                comms = comms + elements.text + " "
                region = region_list[region_count]

            beigebook.loc[len(beigebook), ["Date", "Region", "Sector", "Comments"]] = [date, str(region), sector_name, comms]

https://www.federalreserve.gov/fomc/beigebook/1996/19961030/FullReport.htm
https://www.federalreserve.gov/fomc/beigebook/1996/19961204/FullReport.htm
https://www.federalreserve.gov/fomc/beigebook/1997/19970122/FullReport.htm
https://www.federalreserve.gov/fomc/beigebook/1997/19970312/FullReport.htm
https://www.federalreserve.gov/fomc/beigebook/1997/19970507/FullReport.htm
https://www.federalreserve.gov/fomc/beigebook/1997/19970618/FullReport.htm
https://www.federalreserve.gov/fomc/beigebook/1997/19970806/FullReport.htm
https://www.federalreserve.gov/fomc/beigebook/1997/19970917/FullReport.htm
https://www.federalreserve.gov/fomc/beigebook/1997/19971029/FullReport.htm
https://www.federalreserve.gov/fomc/beigebook/1997/19971203/FullReport.htm
https://www.federalreserve.gov/fomc/beigebook/1998/19980121/FullReport.htm
https://www.federalreserve.gov/fomc/beigebook/1998/19980318/FullReport.htm
https://www.federalreserve.gov/fomc/beigebook/1998/19980506/FullReport.htm
https://www.federalreserv

In [6]:
#Data Cleaning
beigebook["Region"] = beigebook["Region"].str.split("_").apply(lambda x: x[1:])
beigebook["Region"] = beigebook["Region"].apply(lambda x: [s.capitalize() for s in x])
beigebook["Region"] = beigebook["Region"].apply(lambda x: " ".join(x))

Code to scrape the FOMC Meeting Dates

In [7]:
fomc= pd.DataFrame(columns=["Date"])

row = 0

for  year in range(1996, 2018):
     
     master_url = "https://www.federalreserve.gov/monetarypolicy/fomchistorical" + str(year) + ".htm"
     time.sleep(1+0.06*np.random.rand())
     page = requests.get(master_url)
     soup = BeautifulSoup(page.content, 'html.parser')
     h5 = soup.find_all('h5')

     if year>=2011:

        master_div = soup.find_all('div',class_='row divided-row')

     else:

        master_div = soup.find_all('div',class_='row divided-row panel-body')

     for y in master_div:

        results = y.find_all('p' )

        for x in results:

          atags=x.a

          if year<2008:

               if year == 2007:

                  for htags in h5:

                     if "Meeting" in htags.text:

                           if "meeting.pdf" in atags['href']:
                              dates = [atags['href'].split("meeting")[0][-8:]]
                              fomc.loc[row,"Date"] = datetime.strptime(dates[0], "%Y%m%d").date()       
               else:

                  if atags.text =="Minutes":

                     dates = [atags['href'].split(".htm")[0][-8:]]
                     dates[0].strip('[]')

                     fomc.loc[row,"Date"] = datetime.strptime(dates[0], "%Y%m%d").date()

          elif year>=2008:

               if "Minutes" and "Released" in x.text:
                     
                     dates = [atags['href'].split(".htm")[0][-8:]]
                     dates[0].strip('[]')
                     fomc.loc[row,"Date"] = datetime.strptime(dates[0], "%Y%m%d").date()

          row = row + 1  

url = "https://www.federalreserve.gov/monetarypolicy/fomccalendars.htm"
time.sleep(1+0.06*np.random.rand())
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
new = soup.find_all('div',class_='col-xs-12 col-md-4 col-lg-2')

for n in new:

   atagz = n.find_all('a', href = True)

   for a in atagz:
      
      if a.text == 'PDF':
         datez = [a['href'].split("a1")[0][-8:]]
         fomc.loc[row,"Date"] = datetime.strptime(datez[0].strip('[]'), "%Y%m%d").date()
         row += 1
fomc = fomc.drop_duplicates(subset = "Date") #Drops the duplicates
fomc = fomc.sort_values('Date', ascending=True) #Sorts the dates, as from 2018 onwards, dates were descending

#Drop FOMC Meeting Dates that don't have a corresponding beige book or beige book doesn't give sentiment score
fomc['Date'] = pd.to_datetime(fomc['Date'])

drop_dates = ['2020-03-03', '2020-03-23', '2003-09-16']

fomc = fomc[~fomc['Date'].isin(drop_dates)]
#drop dates to match end 1996-end of 2022 range
fomc = fomc.drop(index=fomc.index[:6].union(fomc.index[-2:]))

Sentiment Analysis

In [8]:
# # Section 5: Incorporate a relevant information signal created from textual analysis, e.g., of social media feeds or news sources.
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch
import statistics

model = AutoModelForSequenceClassification.from_pretrained('ProsusAI/finbert', num_labels=3)
tokenizer = AutoTokenizer.from_pretrained('ProsusAI/finbert')

# nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)
text = beigebook.iloc[:, 3]
for index, r in text.iteritems():
    try:
        sentences = str(r)
        inputs = tokenizer(r, return_tensors="pt")
        tokens = tokenizer.tokenize(sentences)
        print('Carrying out sentiment analysis of row no :'index)

        if len(tokens) > 512:
            words = sentences.split()
            mid = len(words)//2
            sent_1 = ' '.join(words[:mid])
            sent_2 = ' '.join(words[mid:])
            input_sent_1 = tokenizer(sent_1,return_tensors="pt")

            input_sent_2 = tokenizer(sent_2,return_tensors="pt")


            tokens1 = tokenizer.tokenize(sent_1)
            tokens2 = tokenizer.tokenize(sent_2)

            with torch.no_grad():
                logits1 = model(**input_sent_1).logits
            scores1 = {k: v for k, v in zip(model.config.id2label.values(), scipy.special.softmax(logits1.numpy().squeeze()))}


            with torch.no_grad():
                logits2 = model(**input_sent_2).logits
            scores2 = {k: v for k, v in zip(model.config.id2label.values(), scipy.special.softmax(logits2.numpy().squeeze()))}

            avg_pos = (((len(tokens1)/(len(tokens1)+len(tokens2)))*scores1['positive']) + ((len(tokens2)/(len(tokens1)+len(tokens2)))*scores2['positive']))
            avg_neg = (((len(tokens1)/(len(tokens1)+len(tokens2)))*scores1['negative']) + ((len(tokens2)/(len(tokens1)+len(tokens2)))*scores2['negative']))
            avg_neu = (((len(tokens1)/(len(tokens1)+len(tokens2)))*scores1['neutral']) + ((len(tokens2)/(len(tokens1)+len(tokens2)))*scores2['neutral']))
            beigebook.loc[index,['Positive','Negative','Neutral']]=[avg_pos,avg_neg,avg_neu]


        else:
            with torch.no_grad():
                logits = model(**inputs).logits
            scores = {k: v for k, v in zip(model.config.id2label.values(), scipy.special.softmax(logits.numpy().squeeze()))}
            beigebook.loc[index,['Positive','Negative','Neutral']]=[scores['positive'],scores['negative'],scores['neutral']]

    except Exception as e:
        print(f"Error occurred at row {index}: {e}")
        continue
        
beigebook['Sentiment_index'] = ((beigebook['Positive'] - beigebook['Negative'])/2) # Adds a column with the Sentiment_index
# # Sentiment analysis i.e. Positive, Negative and Neutral

In [9]:
#Cleaning up the sector column 
(Some cleaning up took place in excel)
sector_keys= pd.read_csv("Sector_keys.csv") 
for index , i in sector_keys.iterrows():
    beigebook['Sector'] = beigebook['Sector'].replace(i[0],i[1])

beigebook['Sector'] = beigebook['Sector'].str.replace("\xa0",'')
beigebook['Sector'] = beigebook['Sector'].str.replace("\n",'')
beigebook = beigebook.dropna()
beigebook = beigebook[beigebook['Sector'] != 'Null']

# Cleaning up the region data
reg_keys= pd.read_csv("Region Keys.csv")
for index , i in reg_keys.iterrows():
    beigebook['Region'] = beigebook['Region'].replace(i[0],i[1])

beigebook['Region'] = beigebook['Region'].str.replace("_",'')
beigebook['Region'] = beigebook['Region'].str.replace("Summary",'')
beigebook = beigebook[beigebook['Sector'] != 'Consumer spending']
beigebook.to_csv('beigebook.csv')

Macro data gathering

In [10]:
#Run the commented-out code below if you want to skip sentiment analysis and carry on with the prviously created dataset produced from it.
# beigebook = pd.read_csv('beigebook.csv')
fred = fa.Fred(api_key='76c20b6988354509c6574aee17017680')
cpi = fred.get_series('CPIAUCSL', observation_start='1996-01-01', observation_end='2023-04-01')
consconf = fred.get_series('CSCICP03USM665S', observation_start='1996-01-01', observation_end='2023-04-01')
ppi = fred.get_series('CSCICP03USM665S', observation_start='1996-01-01', observation_end='2023-04-01')
targetnew = pd.DataFrame(fred.get_series('DFEDTARU', observation_start='1996-01-01', observation_end='2023-04-01'))
targetold = pd.DataFrame(fred.get_series('DFEDTAR', observation_start='1996-01-01', observation_end='2023-04-01'))
sp500 = yf.download("^GSPC", start="1996-01-01", end="2023-04-18")  
sp500.index = sp500.index.date
df1 = pd.concat([targetold, targetnew], axis = 0 ) #Merge older and new Target Rate tickers
df1.columns = ['Target Rate']
df1 = df1.rename_axis('Date')
df1['S&P500'] = sp500['Adj Close']
df2 = pd.DataFrame({'CPI': cpi, 'PPI': ppi, 'Consumer Confidence': consconf})
df2.index.name = 'Date'

[*********************100%***********************]  1 of 1 completed


Section 3: Outline the required steps in cleaning, checking and organisation of the data.

In [11]:

fomc['Date'] = pd.to_datetime(fomc['Date'])
fomc.index = fomc['Date']
# Match Fed Target Rate to the corresponding FOMC meeting date
fomc['Date'] = pd.to_datetime(fomc['Date'])
fomc.index = fomc['Date']
for index1, row1 in fomc.iterrows():
    if fomc.loc[index1, 'Date'] in df1.index:
        fomc.loc[index1, 'Target Rate'] = df1.loc[fomc.loc[index1, 'Date'] , 'Target Rate']
#Create column w the delta of the interest rate
fomc['Delta'] = fomc['Target Rate'].diff()
print(fomc)

                 Date  Target Rate  Delta
Date                                     
1996-11-13 1996-11-13         5.25    NaN
1996-12-17 1996-12-17         5.25   0.00
1997-02-04 1997-02-04         5.25   0.00
1997-03-25 1997-03-25         5.50   0.25
1997-05-20 1997-05-20         5.50   0.00
...               ...          ...    ...
2022-06-15 2022-06-15         1.00   0.50
2022-07-27 2022-07-27         1.75   0.75
2022-09-21 2022-09-21         2.50   0.75
2022-11-02 2022-11-02         3.25   0.75
2022-12-14 2022-12-14         4.00   0.75

[210 rows x 3 columns]


In [12]:
# beigebook = pd.read_csv('Sisyphus.csv')
html_link = pd.concat([html_link, fomc[['Target Rate', 'Delta']].reset_index(drop = True)], axis = 1)
html_link['Date'] = pd.to_datetime(html_link['Date'])
html_link.index = html_link['Date']


In [14]:
#Match target rate with the rows with the corresponding beige book dates
for index, row in beigebook.iterrows():

    if beigebook.loc[index, 'Date'] in html_link.index:
        # print(beigebook.loc[index, 'Date'])
        beigebook.loc[index, 'Target Rate'] = html_link.loc[beigebook.loc[index, 'Date'], 'Target Rate']
        beigebook.loc[index, 'Delta'] = html_link.loc[beigebook.loc[index, 'Date'], 'Delta']
print(beigebook)

       Unnamed: 0       Date          Region  \
0               0 1996-10-30         Overall   
1               1 1996-10-30         Overall   
2               2 1996-10-30         Overall   
3               3 1996-10-30         Overall   
4               4 1996-10-30         Overall   
...           ...        ...             ...   
16474       16985 2022-11-30  San Fransicsco   
16475       16986 2022-11-30  San Fransicsco   
16476       16987 2022-11-30  San Fransicsco   
16477       16988 2022-11-30  San Fransicsco   
16478       16989 2022-11-30  San Fransicsco   

                                  Sector  \
0             Selected Business Services   
1          Consumer Spending and Tourism   
2       Manufacturing and Transportation   
3           Real Estate and Construction   
4      Agriculture and Natural Resources   
...                                  ...   
16474                             Retail   
16475   Manufacturing and Transportation   
16476  Agriculture and Natu

In [13]:
beigebook['Date'] = pd.to_datetime(beigebook['Date'])
print(html_link.index)

DatetimeIndex(['1996-10-30', '1996-12-04', '1997-01-22', '1997-03-12',
               '1997-05-07', '1997-06-18', '1997-08-06', '1997-09-17',
               '1997-10-29', '1997-12-03',
               ...
               '2021-12-01', '2022-01-01', '2022-03-01', '2022-04-01',
               '2022-06-01', '2022-07-01', '2022-09-01', '2022-10-01',
               '2022-11-01',        'NaT'],
              dtype='datetime64[ns]', name='Date', length=210, freq=None)


c:\Users\sksre\anaconda3\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '30/10/1996' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)


In [15]:
# Match macro data with the rows with the corresponding beige book dates
for index, row in beigebook.iterrows():
    date = row['Date']
    if beigebook.loc[index, 'Date'] in df1.index:
        #Code to give next working day if beigebook is released on a weekday
        if beigebook.loc[index,'Date'].weekday() == 5: 
            beigebook.loc[index, 'S&P500'] = df1.loc[beigebook.loc[index, 'Date'] + timedelta(days=2), 'S&P500']
            if np.isnan(df1.loc[beigebook.loc[index, 'Date'] + timedelta(days=1), 'S&P500']):    
                beigebook.loc[index, 'S&P500'] = df1.loc[beigebook.loc[index, 'Date'] + timedelta(days=3), 'S&P500']
        elif beigebook.loc[index,'Date'].weekday() == 6:
            beigebook.loc[index, 'S&P500'] = df1.loc[beigebook.loc[index, 'Date'] + timedelta(days=1), 'S&P500']
        elif np.isnan(df1.loc[beigebook.loc[index, 'Date'], 'S&P500']):
                beigebook.loc[index, 'S&P500'] = df1.loc[beigebook.loc[index, 'Date'] + timedelta(days=1), 'S&P500']
            #in case there is a two day gap that's due to a weekend
        else:
            beigebook.loc[index, 'S&P500'] = df1.loc[beigebook.loc[index, 'Date'], 'S&P500']
    mask = np.logical_and(df2.index.year == date.year, df2.index.month == date.month)
    if mask.any():
        
        beigebook.loc[index, 'CPI'] = df2.loc[mask, 'CPI'].values[0]
        beigebook.loc[index, 'PPI'] = df2.loc[mask, 'PPI'].values[0]
        beigebook.loc[index, 'Consumer Confidence'] = df2.loc[mask, 'Consumer Confidence'].values[0]
print(beigebook)

       Unnamed: 0       Date          Region  \
0               0 1996-10-30         Overall   
1               1 1996-10-30         Overall   
2               2 1996-10-30         Overall   
3               3 1996-10-30         Overall   
4               4 1996-10-30         Overall   
...           ...        ...             ...   
16474       16985 2022-11-30  San Fransicsco   
16475       16986 2022-11-30  San Fransicsco   
16476       16987 2022-11-30  San Fransicsco   
16477       16988 2022-11-30  San Fransicsco   
16478       16989 2022-11-30  San Fransicsco   

                                  Sector  \
0             Selected Business Services   
1          Consumer Spending and Tourism   
2       Manufacturing and Transportation   
3           Real Estate and Construction   
4      Agriculture and Natural Resources   
...                                  ...   
16474                             Retail   
16475   Manufacturing and Transportation   
16476  Agriculture and Natu

In [16]:
#Creates column 'Movement' for the movement of the Target Rate 
c = [(beigebook['Delta'] > 0),
     (beigebook['Delta'] < 0),
     (beigebook['Delta'] == 0)]
values = [1, -1, 0]
beigebook['Movement'] = np.select(c, values)

Querying and Database Creation

In [19]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

# Section 2: DB creation and querying - report data set summary statistics.
con = lite.connect('Beigebook.db')
beigebook.to_sql('Beigebook', con, if_exists='replace', index=False)
cur = con.cursor()
cur.execute("""CREATE TABLE Beigebook (Date date, Region TEXT, Sector TEXT, Comments TEXT, Positive INT, Negative INT, Neutral INT, 
            Sentiment_index INT, Target Rate INT, Delta INT, Movement INT, S&P500 INT, CPI INT, PPI INT, Consumer Confidence INT)""");
cur.execute("INSERT OR IGNORE INTO Beigebook VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);", beigebook)

# ----------------------------------------------------------------------------------------------------------------
#if error just run next line

OperationalError: table Beigebook already exists

In [20]:

# SUMMARY STATS + BOX PLOT OF THE POSITIVES, NEGATIVES AND NEUTRALS ALONE
# Count
query_count = """SELECT COUNT(Positive) AS Total_pos, COUNT(Negative) AS Total_neg, COUNT(Neutral) AS Total_neu
                FROM Beigebook"""
query_observations = pd.read_sql_query(query_count, con)
obs_df = pd.DataFrame({'Positive': query_observations['Total_pos'], 'Negative': query_observations['Total_neg'], 'Neutral': query_observations['Total_neu']})

# Mean: For Positives, Negatives and Neutrals
query_average = """SELECT AVG(Positive) AS Avg_pos, AVG(Negative) AS Avg_neg, AVG(Neutral) AS Avg_neu
                FROM Beigebook"""
query_mean = pd.read_sql_query(query_average, con)
mean_df = pd.DataFrame({'Positive': query_mean['Avg_pos'], 'Negative': query_mean['Avg_neg'], 'Neutral': query_mean['Avg_neu']})

# Median: For Positives, Negatives and Neutrals
query_med = """SELECT Positive, Negative, Neutral
            FROM Beigebook
            ORDER BY Positive, Negative, Neutral ASC"""
query_median = pd.read_sql_query(query_med, con)
positive_median = query_median['Positive'].median()
negative_median = query_median['Negative'].median()
neutral_median = query_median['Neutral'].median()
median_df = pd.DataFrame({'Positive': [positive_median], 'Negative': [negative_median], 'Neutral': [neutral_median]})

# Minimum and maximum: For the Positives, Negatives and Neutrals
query_maxi = """SELECT MAX(Positive) AS Max_pos, MAX(Negative) Max_neg, MAX(Neutral) AS Max_neu
            FROM Beigebook"""
query_max = pd.read_sql_query(query_maxi, con)
max_df = pd.DataFrame({'Positive': query_max['Max_pos'], 'Negative': query_max['Max_neg'], 'Neutral': query_max['Max_neu']})

query_mini = """SELECT MIN(Positive) AS Min_pos, MIN(Negative) AS Min_neg, MIN(Neutral) AS Min_neu
            FROM Beigebook"""
query_min = pd.read_sql_query(query_mini, con)
min_df = pd.DataFrame({'Positive': query_min['Min_pos'], 'Negative': query_min['Min_neg'], 'Neutral': query_min['Min_neu']})

# Standard deviation: For the Positives, Negatives and Neutrals
query_stand_dev = """SELECT Positive, Negative, Neutral
               FROM Beigebook"""
query_std = pd.read_sql_query(query_stand_dev, con)
positive_std = query_std['Positive'].std()
negative_std = query_std['Negative'].std()
neutral_std = query_std['Neutral'].std()
std_df = pd.DataFrame({'Positive': [positive_std], 'Negative': [negative_std], 'Neutral': [neutral_std]})

# Summary Table
sum_mat = pd.concat([obs_df, mean_df, median_df, std_df, max_df, min_df], axis = 0)
sum_mat.index = ['Observations', 'Mean', 'Median', 'Std. Dev.', 'Maximum', 'Minimum']
sum_mat.columns = ['Positive', 'Negative', 'Neutral']
lat_summat = sum_mat.to_latex()
print(lat_summat)

# Section 4: Data visualisation - include nice charts and plots of the data set and, if relevant, some visual analysis.
Positive = beigebook['Positive']
Negative = beigebook['Negative']
Neutral = beigebook['Neutral']

fig = go.Figure()
fig.add_trace(go.Box(y=Positive, name = 'Positive'))
fig.add_trace(go.Box(y=Negative, name = 'Negative'))
fig.add_trace(go.Box(y=Neutral, name = 'Neutral'))
fig.update_layout(title='Box Plot for Sentiment Scores')
fig.show()
# ----------------------------------------------------------------------------------------------------------------

C:\Users\sksre\AppData\Local\Temp\ipykernel_25188\2318602737.py:48: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  lat_summat = sum_mat.to_latex()


\begin{tabular}{lrrr}
\toprule
{} &      Positive &      Negative &       Neutral \\
\midrule
Observations &  16479.000000 &  16479.000000 &  16479.000000 \\
Mean         &      0.299006 &      0.647352 &      0.053642 \\
Median       &      0.125314 &      0.816504 &      0.031778 \\
Std. Dev.    &      0.325211 &      0.339828 &      0.077267 \\
Maximum      &      0.960600 &      0.976279 &      0.930647 \\
Minimum      &      0.006355 &      0.010825 &      0.010152 \\
\bottomrule
\end{tabular}



Query for the sectors


In [21]:
# SUMMARY STATS + BOX PLOT OF THE SECTORS ALONE
# Mean: For Sectors
query_average_sec = """SELECT Sector, AVG(Sentiment_Index) AS Mean_sector
                FROM Beigebook
                GROUP BY Sector"""
query_mean_sec = pd.read_sql_query(query_average_sec, con)
query_mean_sec.columns = ['Mean_sector']
query_mean_sec = query_mean_sec.set_index('Sector')

# Median: For Sectors
query_med_sec = """SELECT Sector, Sentiment_Index AS Median_sector
            FROM Beigebook
            GROUP BY Sector"""
query_median_sec = pd.read_sql_query(query_med_sec, con)
med_sec = query_median_sec.groupby('Sector')['Median_sector'].median().to_frame()
med_sec.columns = ['Median_sector']

# Minimum and maximum: For Sectors
query_maxi_sec = """SELECT Sector, MAX(Sentiment_Index) AS Maximum_sector
            FROM Beigebook
            GROUP BY Sector"""
query_max_sec = pd.read_sql_query(query_maxi_sec, con)
query_max_sec.columns = ['Maximum_sector']
query_max_sec = query_max_sec.set_index('Sector')

query_mini_sec = """SELECT Sector, MIN(Sentiment_Index) AS Minimum_sector
            FROM Beigebook
            GROUP BY Sector"""
query_min_sec = pd.read_sql_query(query_mini_sec, con)
query_min_sec.columns = ['Minimum_sector']
query_min_sec = query_min_sec.set_index('Sector')

# Standard deviation: For Sectors
query_stdev_sec = """SELECT Sector, Sentiment_Index AS Standard_Deviation_sector
        FROM Beigebook"""
query_std_sec = pd.read_sql_query(query_stdev_sec, con)
std_sector = query_std_sec.groupby('Sector')['Standard_Deviation_sector'].std().to_frame()
std_sector.columns = ['Standard_Deviation_sector']

# Summary Table for Sector Statistics
sum_mat_sec = pd.concat([query_mean_sec, med_sec, std_sector, query_max_sec, query_min_sec], axis=1)
sum_mat_sec.index = ['Region']
sum_mat_sec.columns = ['Mean', 'Median', 'Std. Dev.', 'Maximum', 'Minimum']
lat_summatsec = sum_mat_sec.to_latex()
print(lat_summatsec)

ValueError: Length mismatch: Expected axis has 2 elements, new values have 1 elements

Query for the regions

In [ ]:
# SUMMARY STATS + BOX PLOT OF THE REGIONS ALONE
# Mean: For Regions
query_average_reg = """SELECT Region, AVG(Sentiment_Index) AS Mean_region
                FROM Beigebook
                GROUP BY Region"""
query_mean_reg = pd.read_sql_query(query_average_reg, con)
query_mean_reg.columns = ['Mean_region']
query_mean_reg = query_mean_reg.set_index('Region')

# Median: For Regions
query_med_reg = """SELECT Region, Sentiment_Index AS Median_region
            FROM Beigebook
            GROUP BY Region"""
query_median_reg = pd.read_sql_query(query_med_reg, con)
med_reg = query_median_reg.groupby('Region')['Median_region'].median().to_frame()
med_reg.columns = ['Median_region']

# Minimum and maximum: For Regions
query_maxi_reg = """SELECT Region, MAX(Sentiment_Index) AS Maximum_region
            FROM Beigebook
            GROUP BY Region"""
query_max_reg = pd.read_sql_query(query_maxi_reg, con)
query_max_reg.columns = ['Maximum_region']
query_max_reg = query_max_reg.set_index('Region')

query_mini_reg = """SELECT Region, MIN(Sentiment_Index) AS Minimum_region
            FROM Beigebook
            GROUP BY Region"""
query_min_reg = pd.read_sql_query(query_mini_reg, con)
query_min_reg.columns = ['Minimum_region']
query_min_reg = query_min_reg.set_index('Region')

# Standard deviation: For Regions
query_stdev_reg = """SELECT Region, Sentiment_Index AS Standard_Deviation_region
        FROM Beigebook"""
query_std_reg = pd.read_sql_query(query_stdev_reg, con)
std_region = query_std_reg.groupby('Region')['Standard_Deviation_region'].std().to_frame()
std_region.columns = ['Standard_Deviation_region']

# Summary Table for Region Statistics
sum_mat_reg = pd.concat([query_mean_reg, med_reg, std_region, query_max_reg, query_min_reg], axis=1)
sum_mat_reg.index = ['Region']
sum_mat_reg.columns = ['Mean', 'Median', 'Std. Dev.', 'Maximum', 'Minimum']
lat_summatreg = sum_mat_reg.to_latex()
print(lat_summatreg)

# ----------------------------------------------------------------------------------------------------------------

Data Visualisation

In [ ]:
# Section 4: Data visualisation - include nice charts and plots of the data set and, if relevant, some visual analysis.
# Sentiment index plot from 1996 to 2022
import plotly.graph_objs as go
import pandas as pd
sentiment_index = beigebook['Sentiment Index']
date = beigebook['Date']
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=date, y=sentiment_index, mode='markers', 
                         marker=dict(color=sentiment_index, colorscale='Viridis', line_width=1)))
fig1.update_layout(title='Sentiment Index over Time',
                  xaxis_title='Date',
                  yaxis_title='Sentiment Index')
fig1.show()
# ----------------------------------------------------------------------------------------------------------------
# Section 4: Data visualisation - include nice charts and plots of the data set and, if relevant, some visual analysis.
# Target rates plot from 1996 to 2022
target_rate = beigebook.iloc[:, 8]
date = beigebook.iloc[:, 0]
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=date, y=target_rate, mode='lines'))
fig2.update_layout(title='Target Rate over Time',
                  xaxis_title='Date',
                  yaxis_title='Target Rate')
fig2.show()
# ----------------------------------------------------------------------------------------------------------------
# Section 4: Data visualisation - include nice charts and plots of the data set and, if relevant, some visual analysis.
# Positive plot from 1996 to 2022
sent_pos = beigebook['Positive']
date = beigebook['Date']
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=date, y=sent_pos, mode='markers', 
                         marker=dict(color=sent_pos, colorscale='inferno', line_width=1)))
fig3.update_layout(title='Positive Sentiment Score over Time',
                  xaxis_title='Date',
                  yaxis_title='Positive Sentiment Score')
fig3.show()
# Section 4: Data visualisation - include nice charts and plots of the data set and, if relevant, some visual analysis.
# Negative plot from 1996 to 2022
sent_neg = beigebook['Negative']
date = beigebook['Date']
fig4 = go.Figure()
fig4.add_trace(go.Scatter(x=date, y=sent_neg, mode='markers', 
                         marker=dict(color=sent_neg, colorscale='blues', line_width=1)))
fig4.update_layout(title='Negative Sentiment Score over Time',
                  xaxis_title='Date',
                  yaxis_title='Negative Sentiment Score')
fig4.show()

# Section 4: Data visualisation - include nice charts and plots of the data set and, if relevant, some visual analysis.
# Neutral plot from 1996 to 2022
sent_neu = beigebook['Neutral']
date = beigebook['Date']
fig5 = go.Figure()
fig5.add_trace(go.Scatter(x=date, y=sent_neu, mode='markers', 
                         marker=dict(color=sent_neu, colorscale='greys', line_width=1)))
fig5.update_layout(title='Neutral Sentiment Score over Time',
                  xaxis_title='Date',
                  yaxis_title='Neutral Sentiment Score')
fig5.show()

Machine Learning Section

In [ ]:
# Creating training and test data 
df= beigebook
print(df.dtypes)


df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date')
split_date = '2012-01-01'

train_data = df[df['Date'] < split_date]
test_data = df[df['Date'] >= split_date]

# One hot encoding of regions
# one_hot_region = pd.get_dummies(df['Region']) 
# df=pd.concat([df,one_hot_region],axis=1)

# One hot encoding of sectors 
# one_hot_sector = pd.get_dummies(df['Sector']) 
# df=pd.concat([df,one_hot_sector],axis=1)

exclude_cols= ['Movement','Comments','Unnamed: 0','Sector','Region','Date','Delta','PPI','S&P500','Consumer Confidence',"Positive",'Negative' ]
predictors = [column for column in df.columns if column != 'Movement']
predictors =[column for column in df.columns if column not in exclude_cols]


# Splitting the training and testing data
# df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(df[predictors], df['Movement'], test_size=0.99, random_state=15)


#Splitting the train and test data based on date
df_train_x=train_data[predictors]
df_train_y=train_data['Movement']
df_test_x=test_data[predictors]
df_test_y=test_data['Movement']

# Splitting the training and testing data
# df_train_x, df_test_x, df_train_y, df_test_y = train_test_split(df[predictors], df['Movement'], test_size=0.99, random_state=15)


# Random Forest model creation 
RF = RandomForestClassifier()
paramgrid = {
             'n_estimators': [20,100,150,250,300,500],
             'criterion': ['gini', 'entropy'],
             'max_depth': [3, 5, 7, 10, 15],
             }
G_CV = GridSearchCV(RF, param_grid=paramgrid, scoring='f1_macro', n_jobs=-1, cv=5)
G_CV.fit(df_train_x, df_train_y)

# neg_mean_absolute_error
best_params = G_CV.best_params_

print("Best parameters:", best_params)
print("Best Score:",{G_CV.best_score_})

rfc_best = RandomForestClassifier(n_estimators=best_params['n_estimators'], 
                                   criterion=best_params['criterion'], 
                                   max_depth=best_params['max_depth'])

rfc_best.fit(df_train_x, df_train_y)

# Testing the model
results = rfc_best.predict(df_test_x)
RF_prob_est = rfc_best.predict_proba(df_test_x)
print(results)
cm = confusion_matrix(df_test_y, results)
cm_df = pd.DataFrame(cm,
                     index = ['Increase','Constant','Decrease'], 
                     columns = ['Increase','Constant','Decrease'])

# Plotting the feature importance 
features = df_train_x.columns

importances = rfc_best.feature_importances_

indices = np.argsort(importances)
plt.title('Feature Importances')
indices = indices[:20]
plt.barh(range(len(indices)), importances[indices], align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()


#Plotting the confusion matrix
plt.figure(figsize=(5,4))
sns.heatmap(cm_df, annot=True, fmt='d')
plt.title('Confusion Matrix')